In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np




In [2]:
# Membaca dataset dari file CSV menggunakan Pandas
df = pd.read_csv("heart_cleveland_upload.csv")

In [3]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,condition
0,69,1,0,160,234,1,2,131,0,0.1,1,1,0,0
1,69,0,0,140,239,0,0,151,0,1.8,0,2,0,0
2,66,0,0,150,226,0,0,114,0,2.6,2,0,0,0
3,65,1,0,138,282,1,2,174,0,1.4,1,1,0,1
4,64,1,0,110,211,0,2,144,1,1.8,1,0,0,0


In [4]:
df.dtypes

age            int64
sex            int64
cp             int64
trestbps       int64
chol           int64
fbs            int64
restecg        int64
thalach        int64
exang          int64
oldpeak      float64
slope          int64
ca             int64
thal           int64
condition      int64
dtype: object

In [5]:
df.shape

(297, 14)

In [6]:

# Menentukan perangkat yang akan digunakan (CPU atau CUDA)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Definisikan model
class ANN(nn.Module):
    def __init__(self, input_size):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.sigmoid(x)
        return x

# Membuat instance model
input_size = 13  # Sesuaikan dengan jumlah atribut
model = ANN(input_size).to(device)

# Definisikan loss function dan optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Konversi data ke Tensor dan pindahkan ke perangkat yang digunakan
X = df.drop("condition", axis=1).values
y = df["condition"].values

X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1).to(device)

# Melatih model
num_epochs = 296
batch_size = 64
num_samples = X_tensor.shape[0]
num_batches = num_samples // batch_size

for epoch in range(num_epochs):
    for batch in range(num_batches):
        start_idx = batch * batch_size
        end_idx = (batch + 1) * batch_size
        
        inputs = X_tensor[start_idx:end_idx]
        labels = y_tensor[start_idx:end_idx]
        
        # Melakukan forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Melakukan backward pass dan pembaruan bobot
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Mencetak loss pada setiap epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/296], Loss: 0.9442
Epoch [2/296], Loss: 2.3343
Epoch [3/296], Loss: 0.4886
Epoch [4/296], Loss: 1.7860
Epoch [5/296], Loss: 0.4832
Epoch [6/296], Loss: 1.6699
Epoch [7/296], Loss: 0.4354
Epoch [8/296], Loss: 1.1191
Epoch [9/296], Loss: 0.5050
Epoch [10/296], Loss: 0.8981
Epoch [11/296], Loss: 0.5323
Epoch [12/296], Loss: 0.8780
Epoch [13/296], Loss: 0.5392
Epoch [14/296], Loss: 0.8743
Epoch [15/296], Loss: 0.5612
Epoch [16/296], Loss: 0.8154
Epoch [17/296], Loss: 0.5953
Epoch [18/296], Loss: 0.7342
Epoch [19/296], Loss: 0.6267
Epoch [20/296], Loss: 0.6757
Epoch [21/296], Loss: 0.6521
Epoch [22/296], Loss: 0.6460
Epoch [23/296], Loss: 0.6694
Epoch [24/296], Loss: 0.6352
Epoch [25/296], Loss: 0.6701
Epoch [26/296], Loss: 0.6333
Epoch [27/296], Loss: 0.6552
Epoch [28/296], Loss: 0.6341
Epoch [29/296], Loss: 0.6373
Epoch [30/296], Loss: 0.6333
Epoch [31/296], Loss: 0.6242
Epoch [32/296], Loss: 0.6280
Epoch [33/296], Loss: 0.6162
Epoch [34/296], Loss: 0.6183
Epoch [35/296], Loss: 0

In [7]:
# Mengkonversi data pengujian ke Tensor dan memindahkannya ke perangkat yang digunakan
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1).to(device)

# Menjadikan model ke mode evaluasi (tidak ada pembaruan bobot)
model.eval()

# Melakukan prediksi pada data pengujian
with torch.no_grad():
    outputs = model(X_test_tensor)
    predicted_labels = (outputs >= 0.5).float()  # Menggunakan threshold 0.5 untuk klasifikasi biner

# Menghitung metrik evaluasi
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, predicted_labels.cpu())
precision = precision_score(y_test, predicted_labels.cpu())
recall = recall_score(y_test, predicted_labels.cpu())
f1 = f1_score(y_test, predicted_labels.cpu())

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

NameError: name 'X_test' is not defined

In [ ]:
torch.save(model.state_dict(), 'asep1.pth')